# Active Inference cadCAD model

This notebook explores multi-agent active inference simulations by representing agents and their locations in a dictionary format..

## cadCAD Standard Notebook Layout

### 0. Dependencies

In [26]:
import pandas as pd
import numpy as np
from random import normalvariate, random
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from radcad import Model, Simulation, Experiment

from tools.model import ActiveGridference

# Additional dependencies

# For analytics
import itertools

# local utils
import tools.utils as u
from tools.control import construct_policies
import random as rand
from pymdp.maths import softmax

### Initializing Agent Network

In [27]:
grid = list(itertools.product(range(10), repeat=2))
print(grid[grid.index((0,0))])

(0, 0)


In [28]:
# create a dict of agents
agents = {}

# Number of agents
NUMBER_AGENTS = 10

for a in range(NUMBER_AGENTS):
    # create new agent
    agent = ActiveGridference(grid)
    # generate target state
    target = (rand.randint(0,9), rand.randint(0,9))
    # add target state
    agent.get_C(target)
    # all agents start in the same position
    agent.get_D((0, 0))

    agents[a] = agent

print(f"Agents: {agents}")

Agents: {0: <tools.model.ActiveGridference object at 0x22db77b20>, 1: <tools.model.ActiveGridference object at 0x129cf8160>, 2: <tools.model.ActiveGridference object at 0x1904d6460>, 3: <tools.model.ActiveGridference object at 0x13eaef790>, 4: <tools.model.ActiveGridference object at 0x166b9f280>, 5: <tools.model.ActiveGridference object at 0x1985c5df0>, 6: <tools.model.ActiveGridference object at 0x12b136220>, 7: <tools.model.ActiveGridference object at 0x18e475f40>, 8: <tools.model.ActiveGridference object at 0x19c58d7f0>, 9: <tools.model.ActiveGridference object at 0x22db77bb0>}


### 1. State Variables

In [29]:
initial_state = {
    'agents': agents
}

### 2. System Parameters

In [30]:
params = {
    'preferred_state': grid,
    'initial_state': grid,
    'noise': [0.00001, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
}

### 3. Policy Functions

- `get_observation`
- `infer_states`
- `calc_efe`
- `calc_action_posterior`
- `sample_action`
- `calc_next_prior`
- `update_env_state`

In [31]:
def p_actinf(params, substep, state_history, previous_state):
    # State Variables
    agents = previous_state['agents']

    # list of all updates to the agents in the network
    agent_updates = []

    for source, agent in agents.items():

        policies = construct_policies([agent.n_states], [len(agent.E)], policy_len = agent.policy_len)
        # get obs_idx
        obs_idx = grid.index(agent.env_state)

        # infer_states
        qs_current = u.infer_states(obs_idx, agent.A, agent.prior, params['noise'])

        # calc efe
        _G = u.calculate_G_policies(agent.A, agent.B, agent.C, qs_current, policies=policies)

        # calc action posterior
        Q_pi = u.softmax(-_G, params['noise'])
        # compute the probability of each action
        P_u = u.compute_prob_actions(agent.E, policies, Q_pi)
        
        # sample action
        chosen_action = u.sample(P_u)

        # calc next prior
        prior = agent.B[:,:,chosen_action].dot(qs_current) 

        # update env state
        # action_label = params['actions'][chosen_action]

        (Y, X) = agent.env_state
        Y_new = Y
        X_new = X
        # here

        if chosen_action == 0: # UP
            
            Y_new = Y - 1 if Y > 0 else Y
            X_new = X

        elif chosen_action == 1: # DOWN

            Y_new = Y + 1 if Y < agent.border else Y
            X_new = X

        elif chosen_action == 2: # LEFT
            Y_new = Y
            X_new = X - 1 if X > 0 else X

        elif chosen_action == 3: # RIGHT
            Y_new = Y
            X_new = X +1 if X < agent.border else X

        elif chosen_action == 4: # STAY
            Y_new, X_new = Y, X 
            
        current_state = (Y_new, X_new) # store the new grid location
        agent_update = {'source': source,
                        'update_prior': prior,
                        'update_env': current_state,
                        'update_action': chosen_action,
                        'update_inference': qs_current}
        agent_updates.append(agent_update)

    return {'agent_updates': agent_updates}

### 4. State Update Functions

In [32]:
def s_agents(params, substep, state_history, previous_state, policy_input):

    agents_new = previous_state['agents'].copy()

    agent_updates = policy_input['agent_updates']

    if agent_updates != []:
        for update in agent_updates:
            s = update['source']
            agent = agents_new[s]
            update_prior = update['update_prior']
            update_env = update['update_env']
            update_action = update['update_action']
            update_inference = update['update_inference']

            agent.prior = update_prior
            agent.env_state = update_env
            agent.current_action = update_action
            agent.current_inference = update_inference

    return 'agents', agents_new

### 5. Partial State Update Blocks

In [33]:
state_update_blocks = [
    {
        'policies': {
            'p_actinf': p_actinf
        },
        'variables': {
            'agents': s_agents
        }
    }
]

### 6. Configuration

In [34]:
model = Model(
    # Model initial state
    initial_state=initial_state,
    # Model Partial State Update Blocks
    state_update_blocks=state_update_blocks,
    # System Parameters
    params=params
)

### 7. Execution

In [35]:
simulation = Simulation(
    model=model,
    timesteps=10,  # Number of timesteps
    runs=1  # Number of Monte Carlo Runs
)

In [36]:
result = simulation.run()

### 8. Analysis

In [14]:
pd.options.plotting.backend = "plotly"

In [37]:
df = pd.DataFrame(result)
df

,agents,simulation,subset,run,substep,timestep
0,{0: <tools.model.ActiveGridference object at 0...,0,0,1,0,0
1,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,1
2,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,2
3,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,3
4,{0: <tools.model.ActiveGridference object at 0...,0,0,1,1,4
...,...,...,...,...,...,...
1095,{0: <tools.model.ActiveGridference object at 0...,0,99,1,1,6
1096,{0: <tools.model.ActiveGridference object at 0...,0,99,1,1,7
1097,{0: <tools.model.ActiveGridference object at 0...,0,99,1,1,8
1098,{0: <tools.model.ActiveGridference object at 0...,0,99,1,1,9


In [40]:
df['agents'][1099][0].C

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])